In [52]:
#Import
from spotify import Client, OAuth
import pandas as pd

In [53]:
auth = OAuth('f716b10140c04e9997c89e1d094745f6', 'b9aa1c134fe244ebaed7b3cba541f6a7')
auth.request_client_credentials()

In [54]:
spotifyClient = Client(auth)

In [55]:
spotifyAPI = spotifyClient.api

In [24]:
def getTrackAttributes(trackIdList = []):
    trackJson = spotifyAPI.tracks_audio_features(trackIdList)
    trackList = trackJson['audio_features']
    trackList = list(filter(None, trackList))
    return trackList

In [44]:
def getAttributeDF(trackFeatList):
    trackDF = pd.DataFrame(trackFeatList)
    trackDF = trackDF.drop(['analysis_url', 'track_href', 'type', 'uri'], axis=1)
    return trackDF

In [63]:
def getPlaylistTrack(play_id,user_id):
    tracks = spotifyAPI.user_playlist_tracks(playlist_id= play_id, user_id= user_id,fields='items(track.id,track.album.name,track.name,track.artists)')
    return tracks


In [65]:
def trackJSONtoDF(tracks,genre):
    flatData = []

    for i in range(len(tracks['items'])):
        trackObj = {}
        trackObj['id'] = tracks['items'][i]['track']['id']
        trackObj['album'] = tracks['items'][i]['track']['album']['name']
        trackObj['track_name'] = tracks['items'][i]['track']['name']
        trackObj['artist'] = tracks['items'][i]['track']['artists'][0]['name']
        trackObj['genre'] = genre
        flatData.append(trackObj)
    return pd.DataFrame(flatData)

In [60]:
trackJson = getTrackAttributes(['0eUOrB1uVJKQEqZ5EX7pvp','0dA2Mk56wEzDgegdC6R17','11dFghVXANMlKmJXsNCbNl'])
pd.DataFrame(trackJson)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'},
 'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4',
 'id': '3TVXtAsR1Inumwj472S9r4',
 'name': 'Drake',
 'type': 'artist',
 'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}

In [75]:
def pipeLineFunc(play_id,user_id,genre):
    # Fetch playlist tracks
    plTracks = getPlaylistTrack(play_id,user_id)
    
    # Convert to DF
    plTracksDF =  trackJSONtoDF(plTracks,genre)
    
    # Get list of track ids
    trackIds = list(plTracksDF['id'])
    
    # Get tracks features
    trackFeat = getTrackAttributes(trackIds)
    
    # Convert JSON to DF
    trackFeatDF = getAttributeDF(trackFeat)
    
    resultDF = pd.concat([plTracksDF, trackFeatDF], axis=1, join='inner')

    return resultDF
    
    

In [84]:
res = pipeLineFunc('0VEri7WKnnwZ9Eu7RIhzPK','exsorz','metal')


,album,artist,genre,id,track_name,acousticness,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,American Tragedy (Deluxe Edition),Hollywood Undead,metal,0eUOrB1uVJKQEqZ5EX7pvp,Hear Me Now,0.000200,0.457,214267,0.838,0eUOrB1uVJKQEqZ5EX7pvp,0.000102,7,0.8880,-3.794,1,0.0348,85.989,4,0.359
1,The Connection,Papa Roach,metal,29c925OyK03yOmgFXezETO,Give Me Back My Life,0.000295,0.260,238720,0.860,29c925OyK03yOmgFXezETO,0.143000,1,0.1150,-6.902,1,0.0597,169.996,4,0.318
2,The Paramour Sessions (International Version),Papa Roach,metal,5RxCC8ecpIb6W9MVrJNmSF,...To Be Loved,0.004730,0.478,182120,0.977,5RxCC8ecpIb6W9MVrJNmSF,0.000000,8,0.5090,-3.162,1,0.0751,96.047,4,0.482
3,The Paramour Sessions (International Version),Papa Roach,metal,7v43k9XzWpyNfPVNMyhwUE,Roses On My Grave,0.059900,0.355,193387,0.712,7v43k9XzWpyNfPVNMyhwUE,0.000019,9,0.0817,-6.271,0,0.0360,93.958,4,0.186
4,Getting Away With Murder (Explicit Version),Papa Roach,metal,2NcNJC2jVccC1kKIxEXsZH,Scars,0.087500,0.455,208200,0.929,2NcNJC2jVccC1kKIxEXsZH,0.000000,11,0.2260,-3.295,1,0.0494,89.783,4,0.484


,album,id,id
0,American Tragedy (Deluxe Edition),0eUOrB1uVJKQEqZ5EX7pvp,0eUOrB1uVJKQEqZ5EX7pvp
1,The Connection,29c925OyK03yOmgFXezETO,29c925OyK03yOmgFXezETO
2,The Paramour Sessions (International Version),5RxCC8ecpIb6W9MVrJNmSF,5RxCC8ecpIb6W9MVrJNmSF
3,The Paramour Sessions (International Version),7v43k9XzWpyNfPVNMyhwUE,7v43k9XzWpyNfPVNMyhwUE
4,Getting Away With Murder (Explicit Version),2NcNJC2jVccC1kKIxEXsZH,2NcNJC2jVccC1kKIxEXsZH
5,Endgame / Appeal To Reason / Siren Song Of The...,2MNLJCavrcJfNdCYk7IHRV,2MNLJCavrcJfNdCYk7IHRV
6,Endgame / Appeal To Reason / Siren Song Of The...,5uJd3lMR2oQ0e6VDoIvwMR,5uJd3lMR2oQ0e6VDoIvwMR
7,Endgame / Appeal To Reason / Siren Song Of The...,7fAURpMGXojfK8Hihfup7k,7fAURpMGXojfK8Hihfup7k
8,Endgame / Appeal To Reason / Siren Song Of The...,2KOJuUTky6l8bONSInHn2W,2KOJuUTky6l8bONSInHn2W
9,Endgame / Appeal To Reason / Siren Song Of The...,0nRauaGzlpH62t9bAnfv12,0nRauaGzlpH62t9bAnfv12
